# Assignment 3 - Finding Errors in Dataset

## Group 5 - Handling the third dataset of group 2

### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import re

### Loading Dataset and Overview

In [2]:
dataset = pd.read_csv("C:\\Users\\ASUS\\OneDrive\\Desktop\\WS2425\\Applied Statistical Data Analysis\\3rd Week\\Datasets to work on\\2_3_Employee Sample Data.txt", delimiter='\t', encoding='ISO-8859-1')
dataset.head()

,EEID,Full Name,Job Title,Department,Business Unit,Gender,Age,Annual Salary,Bonus %,Country,City
0,E02387,Emily Davis,Sr. Manger,IT,Research & Development,Female,55,"$1,41,604",15%,United States,Seattle
1,E04105,Theodore Dinh,Technical Architect,IT,Manufacturing,Male,59,"$99,975",0%,China,Chongqing
2,E02572,Luna Sanders,Director,Finance,Speciality Products,Female,50,"$1,63,099",20%,United States,Chicago
3,E02832,Penelope Jordan,Computer Systems Manager,IT,Manufacturing,Female,26,"$84,913",7%,United States,Chicago
4,E01639,Austin Vo,Sr. Analyst,Finance,Manufacturing,Male,55,"$95,409",0%,United States,Phoenix


In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186 entries, 0 to 185
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   EEID           186 non-null    object
 1   Full Name      186 non-null    object
 2   Job Title      186 non-null    object
 3   Department     186 non-null    object
 4   Business Unit  186 non-null    object
 5   Gender         185 non-null    object
 6   Age            186 non-null    int64 
 7   Annual Salary  186 non-null    object
 8   Bonus %        185 non-null    object
 9   Country        185 non-null    object
 10  City           186 non-null    object
dtypes: int64(1), object(10)
memory usage: 16.1+ KB


In [4]:
dataset.describe()

,Age
count,186.000000
mean,44.946237
std,22.444341
min,0.000000
25%,34.000000
50%,44.500000
75%,54.000000
max,300.000000


#### As seen above the only integer column is "Age" (Annual Salary and Bonus are also numbers but they contain something that differentiates them from being an integer column). By using the command "describe" we find out that the minimum value is 0 and it doesn't make sense :) Also, the maximum age is set to be 300 which is definitely an error.

### Finding errors related to "Age" column

In [5]:
errors = []
age_issues = dataset[(dataset['Age'] < 18) | (dataset['Age'] > 100)]
if not age_issues.empty:
    errors.append(f"Age Issues:\n{age_issues[['EEID', 'Full Name', 'Age']].to_string(index=False)}\n")

In [6]:
errors

['Age Issues:\n  EEID    Full Name  Age\nE04798   Aurora Ali  300\nE03890 Nevaeh Jones    3\nE02283   Jaxon Park    0\n']

#### As seen above, there are three errors identified in this column. Ages 0, 3, 300 are the anomalies!

### Then we look for errors in the "Bonus" column

In [7]:
bonus_pattern = re.compile(r'^\d{1,2}%$')
bonus_issues = dataset[~dataset['Bonus %'].astype(str).str.strip().apply(lambda x: bool(bonus_pattern.match(x)))]
if not bonus_issues.empty:
    errors.append(f"Bonus Format Issues:\n{bonus_issues[['EEID', 'Full Name', 'Bonus %']].to_string(index=False)}\n")
errors

['Age Issues:\n  EEID    Full Name  Age\nE04798   Aurora Ali  300\nE03890 Nevaeh Jones    3\nE02283   Jaxon Park    0\n',
 'Bonus Format Issues:\n  EEID        Full Name Bonus %\nE00304       Dylan Choi    0.34\nE00586 Sadie Washington     NaN\n']

#### There are two errors. One of them is not in percent mode and the other is null or NaN.

### Then we check if there's anything wrong with the "Gender" column and we count the number of Females and Males (We can also easily look for null cells)

In [8]:
female_count = dataset['Gender'].value_counts().get('Female', 0)
print(f"Number of 'Female' entries in Gender column: {female_count}")

Number of 'Female' entries in Gender column: 98


In [9]:
male_count = dataset['Gender'].value_counts().get('Male', 0)
print(f"Number of 'Male' entries in Gender column: {male_count}")

Number of 'Male' entries in Gender column: 87


#### 98+87=185 but there are 186 rows!

In [10]:
unexpected_gender_rows = dataset[~dataset['Gender'].isin(['Female', 'Male'])]
print("Rows with unexpected Gender values:\n", unexpected_gender_rows)

Rows with unexpected Gender values:
        EEID     Full Name             Job Title Department  \
184  E01037  Jeremiah Chu  IT Systems Architect         IT   

               Business Unit Gender  Age Annual Salary Bonus % Country  \
184  Reasearch & Development    NaN   31      $96,567      0%    China   

         City  
184  Shanghai  


In [11]:
if not unexpected_gender_rows.empty:
    errors.append(f"Gender Issues:\n{unexpected_gender_rows[['EEID', 'Full Name', 'Age']].to_string(index=False)}\n")
errors

['Age Issues:\n  EEID    Full Name  Age\nE04798   Aurora Ali  300\nE03890 Nevaeh Jones    3\nE02283   Jaxon Park    0\n',
 'Bonus Format Issues:\n  EEID        Full Name Bonus %\nE00304       Dylan Choi    0.34\nE00586 Sadie Washington     NaN\n',
 'Gender Issues:\n  EEID    Full Name  Age\nE01037 Jeremiah Chu   31\n']

#### So this single row doesn't have any entry for gender

### Then we check again and we can find out that the country column also has some null cells!

In [12]:
empty_city = dataset.isnull().any()
empty_city

EEID             False
Full Name        False
Job Title        False
Department       False
Business Unit    False
Gender            True
Age              False
Annual Salary    False
Bonus %           True
Country           True
City             False
dtype: bool

In [13]:
null_country_rows = dataset[dataset['Country'].isna()]
print("Rows with null values in 'Country' column:\n", null_country_rows)

Rows with null values in 'Country' column:
       EEID      Full Name         Job Title   Department  \
48  E03720  Dominic Clark  Quality Engineer  Engineering   

             Business Unit Gender  Age Annual Salary Bonus % Country     City  
48  Research & Development   Male   52      $71,476      0%      NaN  Phoenix  


In [14]:
if not null_country_rows.empty:
    errors.append(f"Country Issues:\n{null_country_rows[['EEID', 'Full Name', 'Age']].to_string(index=False)}\n")
errors

['Age Issues:\n  EEID    Full Name  Age\nE04798   Aurora Ali  300\nE03890 Nevaeh Jones    3\nE02283   Jaxon Park    0\n',
 'Bonus Format Issues:\n  EEID        Full Name Bonus %\nE00304       Dylan Choi    0.34\nE00586 Sadie Washington     NaN\n',
 'Gender Issues:\n  EEID    Full Name  Age\nE01037 Jeremiah Chu   31\n',
 'Country Issues:\n  EEID     Full Name  Age\nE03720 Dominic Clark   52\n']

#### As presented above, one cell was missing with the country information and we added it to the our errors. 7 errors by far!

### Trying to find errors in "City" column

In [15]:
expected_cities = ["Seattle", "Chongqing", "Chicago", "Phoenix", "Austin", "Miami", 
                   "Shanghai", "Columbus", "Manaus", "Beijing", "Chengdu", "Rio de Janeiro", 
                   "Sao Paulo"]
empty_city = dataset['City'].isna()
unexpected_cities = dataset[~dataset['City'].isin(expected_cities) & ~empty_city]
print("\nEmpty cells in 'City' column:\n", dataset[empty_city])
print("\nRows with unexpected values in 'City' column:\n", unexpected_cities)


Empty cells in 'City' column:
 Empty DataFrame
Columns: [EEID, Full Name, Job Title, Department, Business Unit, Gender, Age, Annual Salary, Bonus %, Country, City]
Index: []

Rows with unexpected values in 'City' column:
        EEID        Full Name          Job Title   Department  \
22   E03484     Logan Rivera           Director           IT   
26   E04545    Abigail Mejia   Quality Engineer  Engineering   
88   E00431  Eliza Hernandez  Network Architect           IT   
134  E02464     Sophie Silva     Vice President  Engineering   
137  E02783      Levi Mendez     Vice President        Sales   

              Business Unit  Gender  Age Annual Salary Bonus % Country  \
22   Research & Development    Male   59    $1,72,787     28%   Brazil   
26                Corporate  Female   56      $98,581      0%   Brazil   
88                Corporate  Female   48      $76,588      0%   Brazil   
134               Corporate  Female   28    $2,40,488     40%   Brazil   
137  Research & Develo

#### It seems that these 5 typos in the name of "Rio de Janeiro" has been there at the begining and it's not deliberately done by someone :D

### Looking for special characters in the dataset

In [16]:
def contains_special_characters(s):
    if isinstance(s, str):
        return bool(re.search(r'[^a-zA-Z0-9 %,.$&]', s))
    return False
special_chars_mask = dataset.map(contains_special_characters)
rows_with_special_chars = dataset[special_chars_mask.any(axis=1)]
if not rows_with_special_chars.empty:
    print("Rows with special characters:")
    print(rows_with_special_chars)
else:
    print("No rows with special characters found.")

Rows with special characters:
       EEID     Full Name              Job Title Department  \
15   E00549   Isabella Xi         Vice President  Marketing   
112  E04735  Nova Coleman  System Administratorÿ         IT   

              Business Unit  Gender  Age Annual Salary Bonus %        Country  \
15   Research & Development  Female   41    2,49,270 ?    30%   United States   
112           Manufacturing  Female   44      $74,738      0%   United States   

        City  
15   Seattle  
112    Miami  


#### As it is shown, two rows have special characters. One of them has a "ÿ" letter and the other has a question mark next to the salary digits!

In [17]:
if not rows_with_special_chars.empty:
    errors.append(f"Special Characters:\n{rows_with_special_chars[['EEID', 'Full Name', 'Age']].to_string(index=False)}\n")
errors

['Age Issues:\n  EEID    Full Name  Age\nE04798   Aurora Ali  300\nE03890 Nevaeh Jones    3\nE02283   Jaxon Park    0\n',
 'Bonus Format Issues:\n  EEID        Full Name Bonus %\nE00304       Dylan Choi    0.34\nE00586 Sadie Washington     NaN\n',
 'Gender Issues:\n  EEID    Full Name  Age\nE01037 Jeremiah Chu   31\n',
 'Country Issues:\n  EEID     Full Name  Age\nE03720 Dominic Clark   52\n',
 'Special Characters:\n  EEID    Full Name  Age\nE00549  Isabella Xi   41\nE04735 Nova Coleman   44\n']

### In total 9 errors were found!